In [94]:
from bs4 import BeautifulSoup
import requests
from pprint import pprint
import pandas as pd
import datetime as dt
from tqdm import tqdm
import functools

## Break

In [168]:
df = pd.read_csv('unc_dining_with_counts.csv')
df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d") # does pandas implicity convert date but not time?
df['period_start'] = pd.to_datetime(df['period_start'], format="%H:%M:%S").dt.time
df['period_end'] = pd.to_datetime(df['period_end'], format="%H:%M:%S").dt.time
df

    


,Unnamed: 0,date,location,period,station,item,period_start,period_end,item_count,item_prob
0,0,2023-09-19,Chase,Breakfast (7am-10:45am),Waffle Bar,Waffle Bar,07:00:00,10:45:00,721,1.000000
1,1,2023-09-19,Chase,Breakfast (7am-10:45am),Specialty Bakery,Lemon Poppyseed Muffin,07:00:00,10:45:00,33,0.045770
2,2,2023-09-19,Chase,Breakfast (7am-10:45am),Specialty Bakery,Vegan Coffee Cake,07:00:00,10:45:00,36,0.049931
3,3,2023-09-19,Chase,Breakfast (7am-10:45am),Bakery,Mini Donuts,07:00:00,10:45:00,152,0.210818
4,4,2023-09-19,Chase,Breakfast (7am-10:45am),Bakery,Chocolate Soft Serve Ice Cream,07:00:00,10:45:00,454,0.629681
...,...,...,...,...,...,...,...,...,...,...
145246,145246,2023-12-16,Chase,Dinner (5pm-8pm),Beverages,Cheerwine®,17:00:00,20:00:00,717,0.994452
145247,145247,2023-12-16,Chase,Dinner (5pm-8pm),Beverages,Pepsi® Zero Sugar,17:00:00,20:00:00,717,0.994452
145248,145248,2023-12-16,Chase,Dinner (5pm-8pm),Beverages,Pepsi®,17:00:00,20:00:00,717,0.994452
145249,145249,2023-12-16,Chase,Dinner (5pm-8pm),Beverages,Gatorade® Fruit Punch,17:00:00,20:00:00,717,0.994452


In [172]:
date = (dt.datetime.today() + dt.timedelta(days=1)).date()
date

datetime.date(2023, 11, 21)

In [177]:
df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d").dt.date

df[df['date']==date]
print('TESTTTTTTTTT', type(df['date'].iloc[0]), type(date))

NameError: name 'self' is not defined

In [170]:
df['date']

0        2023-09-19
1        2023-09-19
2        2023-09-19
3        2023-09-19
4        2023-09-19
            ...    
145246   2023-12-16
145247   2023-12-16
145248   2023-12-16
145249   2023-12-16
145250   2023-12-16
Name: date, Length: 145251, dtype: datetime64[ns]

In [145]:
from fuzzywuzzy import fuzz


class Menu:
    cache = {}

    intresting_stations = [
        'The Griddle',
        'The Kitchen Table',
        'Rotisserie',
        'International Flavors ',
        'Homemade Soups & Sushi',
        'International Flavors',
        'Soup and Salads',
        'Simply Prepared Grill',
        'Specialty Bakery',
    ]

    interesting_food = [ # (exclude, reinclude_list)
        ('Sauce', ['in', 'with']),
        ('Dip', ['in', 'with']),
    ]

    def __init__(self, df = None):
        if df is None:
            df = self.fetch()
        else:
            self.df = df

    @staticmethod
    def fetch(self):
        start_date = dt.datetime.today().strftime('%Y-%m-%d')
        df = get_menu(pd.date_range(start=start_date, end='2023-12-20'))
        df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
        df['period_start'] = pd.to_datetime(df['period_start'], format="%I:%M%p").dt.time
        df['period_end'] = pd.to_datetime(df['period_end'], format="%I:%M%p").dt.time
        df = df.join(
            df.drop(columns=['station']).drop_duplicates()['item'].value_counts(),
            on='item',
            how='left',
            rsuffix='_count'
        )
        df['item_prob'] = df['item_count'] / len(df[['date', 'location', 'period']].drop_duplicates())
        return Menu(df)

    def filter_date(self, date = dt.datetime.today()):
        if date == 'tommorow':
            date = (dt.datetime.today() + dt.timedelta(days=1))
        elif date == 'today':
            date = dt.datetime.today()

        date = date.strftime('%Y-%m-%d')
        return Menu(self.df[self.df['date'] == date])

    def filter_location(self, location):
        return Menu(self.df[self.df['location'] == location])
    
    def filter_time(self, time = dt.datetime.now().time()):
        if time == 'lunch':
            time = dt.time(13)
        if time == 'dinner':
            time = dt.time(18)

        return Menu(self.df[
            (self.df['period_start'] <= time) & (
                (self.df['period_end'] >= time) | (self.df['period_end'] == dt.time(0))
            )])

    def filter_common(self, threshold = 0.2):
        return Menu(self.df[self.df['item_prob'] < threshold])
    
    def filter_item(self, item, exact=False):
        if exact:
            return Menu(self.df[self.df['item'] == item])
        else:
            search_list = list(self.df['item'].unique())
            search_list.sort(key=lambda x: fuzz.ratio(x, item), reverse=True)
            return search_list[0], Menu(self.df[self.df['item'] == search_list[0]])


    def filter_generic(self, column, f):
        return Menu(self.df[f(self.df[column])])
    
    def sort_values(self, *args, **kwargs):
        return Menu(self.df.sort_values(*args, **kwargs))

    def use_cache(**expiry):
        def partial(method):
            def wrapper(self, *args, **kwargs):
                key = (method.__name__)

                if key in Menu.cache \
                    and Menu.cache[key]['expiry'] > dt.datetime.now():
                    print(f'Using cache for {key}, TTL: {Menu.cache[key]["expiry"] - dt.datetime.now()}')
                    return Menu.cache[key]['data']
                
                out = method(self, *args, **kwargs)
                Menu.cache[key] = {
                    'data': out,
                    'expiry': dt.datetime.now() + dt.timedelta(**expiry)
                }
                return out
            return wrapper
        return partial

    @use_cache(days=1)
    def get_daily_overview(self, date=dt.datetime.today()):
        highlights = {}
        for location in self.df['location'].unique():
            for time in ['lunch', 'dinner']:
                highlight = menu \
                    .filter_common() \
                    .filter_generic('station', lambda x: x.isin(Menu.intresting_stations)) \
                    .filter_generic('item_prob', lambda p: p < 0.2) \
                    .filter_generic('item', lambda x: x.str.contains('Sauce|Dip', case=False, regex=True) == False) \
                    .sort_values(by=['item_prob'], ascending=True) \
                    .filter_date(date) \
                    .filter_time(time) \
                    .filter_location(location)[:8]
                highlight = highlight['item'] + highlight['item_prob'].apply(lambda x: f' ({x*100:.0f}% Rarity)')
                highlight = '\n'.join(highlight)
                
                highlights[(location, time)] = highlight

        out_lunch = f'''
\U0001f306 <u><b>Overview for {dt.datetime.today().strftime("%A, %d %b")} Lunch</b></u>:

\U0001f3df <b>Chase Lunch</b>:
{highlights[('Chase', 'lunch')]}

\U0001f3db <b>Lenoir Lunch</b>:
{highlights[('Lenoir', 'lunch')]}
'''
        out_dinner = f'''
\U0001f306 <u><b>Overview for {dt.datetime.today().strftime("%A, %d %b")} Dinner</b></u>:

\U0001f3df <b>Chase Dinner</b>:
{highlights[('Chase', 'dinner')]}

\U0001f3db <b>Lenoir Dinner</b>:
{highlights[('Lenoir', 'dinner')]}
'''   
        return out_lunch, out_dinner

    def __repr__(self):
        return self.df.__repr__()
    
    def _repr_html_(self):
        return self.df._repr_html_ ()
    
    def __getitem__(self, i):
         return self.df[i]
    
menu = Menu(df)

menu.get_daily_overview()

C:\Users\Jing Wen\AppData\Roaming\Python\Python39\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


('\n🌆 <u><b>Overview for Monday, 20 Nov Lunch</b></u>:\n\n🏟 <b>Chase Lunch</b>:\nApple Braised Cabbage (0% Rarity)\nBratwurst (0% Rarity)\nCajun Chicken Breast  (1% Rarity)\nChicken & Wild Rice Soup  (1% Rarity)\nCurried Cauliflower Soup (1% Rarity)\nVegan Caramel Apple Trifle (2% Rarity)\nCurry Chicken (2% Rarity)\nRoasted Garlic Green Beans (2% Rarity)\n\n🏛 <b>Lenoir Lunch</b>:\nMushrooms with Red Onions (0% Rarity)\nGrilled Lemon Chicken (0% Rarity)\nBlackened Shrimp and Broccoli Alfredo (0% Rarity)\nMediterranean Style Cauliflower (0% Rarity)\nTurkey & Sausage Black Bean Chili (1% Rarity)\nHerbed Tomato Ratatouille (1% Rarity)\nVegan Caramel Apple Trifle (2% Rarity)\nPeanut Butter Blondie (2% Rarity)\n',
 '\n🌆 <u><b>Overview for Monday, 20 Nov Dinner</b></u>:\n\n🏟 <b>Chase Dinner</b>:\nSweet Potato Mash (1% Rarity)\nShrimp Fried Rice  (1% Rarity)\nBaby Carrots with Fresh Dill (1% Rarity)\nSavory Grits (1% Rarity)\nCurried Cauliflower Soup (1% Rarity)\nBaked Salmon (1% Rarity)\nChic

In [147]:
menu = Menu(df)

menu.filter_item('Chicken Florentine')

('Chicken Florentine Soup',
              date location             period                 station  \
 18199  2023-09-29   Lenoir   Lunch (11am-3pm)  Homemade Soups & Sushi   
 19358  2023-09-30   Lenoir  Brunch (11am-3pm)  Homemade Soups & Sushi   
 43063  2023-10-13   Lenoir   Lunch (11am-3pm)  Homemade Soups & Sushi   
 44236  2023-10-14   Lenoir  Brunch (11am-3pm)  Homemade Soups & Sushi   
 64612  2023-10-27   Lenoir   Lunch (11am-3pm)  Homemade Soups & Sushi   
 65793  2023-10-28   Lenoir  Brunch (11am-3pm)  Homemade Soups & Sushi   
 89614  2023-11-10   Lenoir   Lunch (11am-3pm)  Homemade Soups & Sushi   
 90786  2023-11-11   Lenoir  Brunch (11am-3pm)  Homemade Soups & Sushi   
 131819 2023-12-08   Lenoir   Lunch (11am-3pm)  Homemade Soups & Sushi   
 132990 2023-12-09   Lenoir  Brunch (11am-3pm)  Homemade Soups & Sushi   
 
                            item period_start period_end  item_count  item_prob  
 18199   Chicken Florentine Soup     11:00:00   15:00:00          10    0.